# Non-Uniform Cubic Hermite Splines

Uniform spline: each segment is defined on the same unit interval, regardless how far apart (spatially) their start and end points are.
Longer elements show a higher velocity.
But the tangent vectors can be defined in a way that the velocity is smoothly changing from one segment to the next ($C^1$ continuity).

Idea: just use a larger time interval for longer segments?
But this doesn't work!
If we make the incoming and outgoing tangent vector the same, there is a sudden change in speed at the segment border ($G^1$ continuity):

TODO: image

For non-unit intervals we'll have to go back to the drawing board!
It will turn out that the size of the time interval $\Delta_0$ has to scale the tangent vectors for this to keep working.

Plan: substitute $t \to \frac{t - t_0}{t_1 - t_0}$ and do the same as for uniform Hermine splines.

In [ ]:
%matplotlib inline
from IPython.display import display

In [ ]:
import sympy as sp
sp.init_printing(order='rev-lex')

In [ ]:
from utility import NamedExpression, NamedMatrix

In [ ]:
t, t0, t1 = sp.symbols('t t:2')

In [ ]:
delta = t1 - t0, sp.S('Delta0')
sp.Eq(*delta)

In [ ]:
coefficients = sp.Matrix(sp.symbols('abm:4')[::-1])
b_monomial = sp.Matrix([t**3, t**2, t, 1]).T
b_monomial.dot(coefficients)

In [ ]:
p = _.subs(t, (t - t0) / (t1 - t0))
p.subs(*delta)

In [ ]:
velocity = p.diff(t)
velocity.subs(*delta)

The start and end point are the same as in the uniform case, since the amount of time in the interval doesn't affect them:

In [ ]:
x0 = NamedExpression('xbm0', p.subs(t, t0))
x1 = NamedExpression('xbm1', p.subs(t, t1))
display(x0, x1)

The velocities are similar to the uniform case, but they are divided by the interval $\Delta_0 = t_1 - t_0$:

In [ ]:
xd0 = NamedExpression('xdotbm0', velocity.subs(t, t0))
xd1 = NamedExpression('xdotbm1', velocity.subs(t, t1).factor())
display(xd0.subs(*delta), xd1.subs(*delta))

As before, we can now try to extract the matrix values from the equations:

In [ ]:
M_H = NamedMatrix(r'{M_\text{H}}', 4, 4)

In [ ]:
control_values_H = NamedMatrix(sp.Matrix([x0.name, x1.name, xd0.name, xd1.name]),
                               M_H.name.I * coefficients)
control_values_H

In [ ]:
substitutions = x0, x1, xd0, xd1

In [ ]:
control_values_H.subs(substitutions).subs(*delta)

In [ ]:
M_H.I = sp.Matrix([[expr.expand().coeff(c) for c in coefficients]
                   for expr in control_values_H.subs(substitutions).name])
M_H.I.subs(*delta)

In [ ]:
print(_.expr)

In [ ]:
M_H.factor().subs(*delta)

In [ ]:
print(_.expr)

In [ ]:
b_H = NamedMatrix(r'{b_\text{H}}', b_monomial * M_H.expr)
b_H.factor().subs(*delta).simplify().T

So those are the new basis functions.
They are the same as in the uniform case, except that the last two (the ones corresponding to the the tangent vectors at the start and the end) are scaled by $\Delta_0$.

With non-uniform splines, the coefficients are different for each segment,
because generally the values $\Delta_i$ are different.

## Plotting Non-Uniform Hermite Splines

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def hermite_curve(control_values, begin, end):
    coeffs = sp.lambdify([t0, t1], M_H.expr)(begin, end) @ control_values
    
    def evaluate(t):
        t = np.expand_dims(t, -1)
        t = (t - begin) / (end - begin)
        return t**[3, 2, 1, 0] @ coeffs

    return evaluate

In [ ]:
def plot_hermite(control_values, t_values, ax=None):
    if ax is None:
        ax = plt.gca()
    control_values = np.asarray(control_values)
    c = hermite_curve(control_values, t_values[0], t_values[-1])
    xyuv = np.column_stack(np.vsplit(control_values, 2))
    # Move beginning of second arrow:
    xyuv[1, :2] -= xyuv[1, 2:]
    ax.quiver(*xyuv.T, angles='xy', scale_units='xy', scale=1)
    ax.scatter(*c(t_values).T)
    ax.axis('equal')

In [ ]:
t_values = np.linspace(0, 1, 15)

For unit intervals, it's the same as before:

In [ ]:
plot_hermite([[0, 0], [5, 2], [1, 2], [1, -2]], t_values)
plot_hermite([[5, 2], [6, 1], [1, -2], [5, 1]], t_values)

In [ ]:
plot_hermite([[0, 0], [5, 2], [1, 2], [1, -2]], np.linspace(0, 3, 45, endpoint=True))
plot_hermite([[5, 2], [6, 1], [1, -2], [5, 1]], np.linspace(0, 2/3, 10, endpoint=True))